In [ ]:
import json
import os
import re
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Folders of interest
base_dir='zscores'
folders_of_interest = ['climate', 'math', 'misinformation_health','gwarming']
#folders_of_interest = ['climate']
emotions = ['anger', 'trust', 'surprise', 'disgust', 'joy', 'sadness', 'fear', 'anticipation']

In [ ]:
output = []

In [ ]:
def extract_model(filename):
    match = re.search(r'_(.*?)_', filename)
    if match:
        return match.group(1)
    return None


In [ ]:
#Consider all topics
for folder in folders_of_interest:
    folder_path = os.path.join(base_dir, folder)

    #Consider all models
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r') as file:
            data = json.load(file)
            for emotion in data.keys():
                pos_count = len([num for num in data[emotion] if num > 1.96])
                neg_count = len([num for num in data[emotion] if num < -1.96])
                output.append({'topic': folder, 'model': extract_model(filename), 'emotion': emotion, 'positive': pos_count, 'negative': neg_count})

In [ ]:
df = pd.DataFrame(output)
display(df)

In [ ]:
model_order = [
    'GPT-3.5',
    'GPT-3.5(ITA)',
    'Haiku',
    'Haiku(ITA)',
    'Mistral-7b',
    'Llama-3-8B',
    'LLaMAntino-2(ITA)'
]

# Create a dictionary for sorting
sorter = {model: i for i, model in enumerate(model_order)}

# Define a function to map the models to their sort order
def model_sort_order(model):
    return sorter.get(model, len(sorter))

# Sort the DataFrame
df = df.sort_values(by='model', key=lambda x: x.map(model_sort_order))


In [ ]:
df['model'].unique()

In [ ]:
#df.to_csv('emotionspresence.csv', index=False)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

colors = {
    'anger': 'orangered',
    'trust': 'olivedrab',
    'surprise': 'skyblue',
    'disgust': 'slateblue',
    'joy': 'gold',
    'fear': 'forestgreen',
    'sadness': 'dodgerblue',
    'anticipation': 'darkorange'
}
# Dictionary to map topics to titles
topic_titles = {
    'climate': 'Climate Change',
    'math': 'Math Anxiety',
    'misinformation_health': 'Misinformation in Health',
    'gwarming': 'Global Warming'
}




# Iterate over each topic
for topic in topic_titles.keys():
    # Filter data for the current topic
    topic_data = df[df['topic'] == topic]

    # Create a new figure for presence of emotions
    fig_presence, axs_presence = plt.subplots(1, 8, figsize=(10, 3),sharey=True,sharex=True)
    fig_presence.suptitle(f'{topic_titles[topic]} - Presence of emotionally-charged texts.', fontsize=20, ha='center', fontweight='bold')

    # Create a new figure for lack of emotions
    fig_negative, axs_negative = plt.subplots(1, 8, figsize=(10, 3),sharey=True,sharex=True)
    fig_negative.suptitle(f'{topic_titles[topic]} - Lack of emotionally-charged texts.', fontsize=20, ha='center', fontweight='bold')

    # Iterate over each emotion
    for i, emotion in enumerate(emotions):
        # Filter data for the current emotion
        emotion_data = topic_data[topic_data['emotion'] == emotion]
        # Get unique models for the current emotion
        models = emotion_data['model'].unique()

        # Set the width of each bar
        bar_width = 0.9
        ## Set the positions of the bars on the x-axis
        #r1 = range(len(models))

        # Create the bar plot for presence of emotions
        axs_presence[i].bar(models, (emotion_data['positive']/ 10), width=bar_width, color=colors[emotion])

        # Create the bar plot for lack of emotions
        axs_negative[i].bar(models, (emotion_data['negative']/ 10), width=bar_width, color=colors[emotion])

        ## Set the x-tick labels and positions

        axs_presence[i].set_xticks(range(len(models)), models, rotation=50, ha='right',rotation_mode='anchor')
        
        axs_negative[i].set_xticks(range(len(models)), models, rotation=50, ha='right',rotation_mode='anchor')


        if i == 0:
            axs_presence[i].set_ylabel('Percentage of emotionally-charged texts')
            axs_negative[i].set_ylabel('Percentage of emotionally-charged texts')
        else:
            axs_presence[i].tick_params(labelleft=False)
            axs_negative[i].tick_params(labelleft=False)
#
        axs_presence[i].set_title(f'{emotion.capitalize()}', fontsize=16, ha='center', fontweight='bold')
        axs_negative[i].set_title(f'{emotion.capitalize()}', fontsize=16, ha='center', fontweight='bold')
#
        # Set the y-axis limits
        axs_presence[i].set_ylim(0, 100)
        axs_negative[i].set_ylim(0, 100)

        # Add transparent grid lines on the y-axis at 20, 40, 60, 80
        axs_presence[i].yaxis.grid(True, which='major', linestyle='-', linewidth='0.5', color='black', alpha=0.3)
        axs_presence[i].set_yticks([20, 40, 60, 80, 100])
        axs_negative[i].yaxis.grid(True, which='major', linestyle='-', linewidth='0.5', color='black', alpha=0.3)
        axs_negative[i].set_yticks([20, 40, 60, 80, 100])

        # Remove top and right borders
        axs_presence[i].spines['top'].set_visible(False)
        axs_presence[i].spines['right'].set_visible(False)
        axs_negative[i].spines['top'].set_visible(False)
        axs_negative[i].spines['right'].set_visible(False)

        # Remove left border for all plots except the first in the row
        if i != 0:
            axs_presence[i].spines['left'].set_visible(False)
            axs_negative[i].spines['left'].set_visible(False)

        # Hide y-axis ticks for all plots except the first in the row
        axs_presence[i].tick_params(bottom=True)
        axs_negative[i].tick_params(bottom=True)
        if i == 0:
            axs_presence[i].tick_params(left=True)
            axs_negative[i].tick_params(left=True)

        

        

    # Adjust the layout and display the plots

    fig_presence.tight_layout()
    fig_negative.tight_layout()
    fig_presence.subplots_adjust(top=0.80,hspace=0, wspace=0.3)  # Adjust top to make room for the centered title
    fig_negative.subplots_adjust(top=0.80,hspace=0, wspace=0.3)  # Adjust top to make room for the centered title
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set_style("white")

colors = {
    'anger': 'orangered',
    'trust': 'olivedrab',
    'surprise': 'skyblue',
    'disgust': 'slateblue',
    'joy': 'gold',
    'fear': 'forestgreen',
    'sadness': 'dodgerblue',
    'anticipation': 'darkorange'
}

topic_titles = {
    'climate': 'Climate Change',
    'gwarming': 'Global Warming',
    'math': 'Math Anxiety',
    'misinformation_health': 'Misinformation in Health'
}

# Iterate over each topic
for topic in topic_titles.keys():
    topic_data = df[df['topic'] == topic]

    fig, axs = plt.subplots(1, 8, figsize=(14, 4.5), sharey=True, sharex=True)
    fig.suptitle(f'{topic_titles[topic]} - Presence or Lack of emotions', fontsize=20, ha='center', fontweight='bold')

    # Iterate over each emotion
    for i, emotion in enumerate(colors.keys()):
        emotion_data = topic_data[topic_data['emotion'] == emotion]
        models = emotion_data['model'].unique()

        bar_width = 0.5
        r1 = np.arange(len(models))
        r2 = [x + bar_width for x in r1]

        # Plotting the positive bars
        axs[i].bar(r1, emotion_data['positive']/ 10, width=bar_width, color=colors[emotion], label='Positive')

        # Plotting the negative bars with white fill and colored edges
        axs[i].bar(r2, emotion_data['negative']/ 10, width=bar_width, color='white', edgecolor=colors[emotion], linewidth=2, label='Negative')

        axs[i].set_xticks([r + bar_width/2 for r in range(len(models))], models, rotation=50, ha='right', rotation_mode='anchor')
        axs[i].set_title(emotion.capitalize(), fontsize=16, ha='center', fontweight='bold')

        if i == 0:
            axs[i].set_ylabel('Percentage of emotionally-charged texts')
        else:
            axs[i].tick_params(labelleft=False)

        axs[i].set_ylim(0, 100)
        axs[i].yaxis.grid(True, which='major', linestyle='-', linewidth='0.5', color='black', alpha=0.3)
        axs[i].set_yticks([20, 40, 60, 80, 100])

        axs[i].spines['top'].set_visible(False)
        axs[i].spines['right'].set_visible(False)
        if i != 0:
            axs[i].spines['left'].set_visible(False)

        axs[i].tick_params(bottom=True)
        if i == 0:
            axs[i].tick_params(left=True)

    fig.tight_layout()
    from matplotlib.lines import Line2D
    from matplotlib.patches import Patch

    legend_elements = [Patch(facecolor='black', edgecolor='black',
                         label='Presence'),
                   Patch(facecolor='white', edgecolor='black',
                         label='Lack')]

    legend = fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.93, 1))




    fig.subplots_adjust(top=0.81, hspace=0, wspace=0.15)
    plt.show()
